In [1]:
import pandas as pd

import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, LabelSet

In [2]:
df=pd.read_csv('subreddits_norm_toxic.csv')
df=df.drop(columns='Unnamed: 0')
df=df.drop_duplicates(subset=['SOURCE_SUBREDDIT','TARGET_SUBREDDIT'])
df['positive_rate']=1-df['toxicity']

In [3]:
df.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,negative,positive,Sum,toxicity,source_parent_group,source_child_group,target_parent_group,target_child_group,norm_toxicity,positive_rate
0,2007scape,askreddit,2,56,58,0.034483,video games,runescape,discussion,ask,0.044828,0.965517
1,accounting,askreddit,1,19,20,0.050000,hobbies/occupations,general,discussion,ask,0.065000,0.950000
2,actuallesbians,askreddit,2,11,13,0.153846,lifestyle,lgbt,discussion,ask,0.200000,0.846154
3,advice,askreddit,1,17,18,0.055556,discussion,advice,discussion,ask,0.072222,0.944444
4,ama,askreddit,1,14,15,0.066667,discussion,ama,discussion,ask,0.086667,0.933333


# Option 1

In [59]:
def radar_graph(subreddit):
    
    ####DATA SELECTION####
    a=df[df['SOURCE_SUBREDDIT']==subreddit]
    a=a.sort_values('toxicity')
    
    
    #TOP 3 friends
    friend_names=list(a['TARGET_SUBREDDIT'][:3])
    
    #TOP 3 enemies
    enemies_names=list(a['TARGET_SUBREDDIT'][-3:])
    
    #TOXICITY
    friend_toxicity=list(a['toxicity'][:3])
    enemies_toxicity=list(a['toxicity'][-3:])
    
    #POSITIVITY
    friends_positivity=list(a['positive_rate'][:3])
    enemies_positivity=list(a['positive_rate'][-3:])
    
    
    #######RADAR_GRAPH creation####
    
    nothing=[""]
    
    #number of vertices
    num_vars = 6

    centre = 0.5

    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2

    def unit_poly_verts(theta, centre ):
        """Return vertices of polygon for subplot axes.
        This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
        """
        x0, y0, r = [centre ] * 3
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts

    def radar_patch(r, theta, centre ):
        """ Returns the x and y coordinates corresponding to the magnitudes of 
        each variable displayed in the radar plot
        """
        # offset from centre of circle
        offset = 0.0
        yt = (r*centre + offset) * np.sin(theta) + centre 
        xt = (r*centre + offset) * np.cos(theta) + centre 
        return xt, yt

    verts = unit_poly_verts(theta, centre)
    x = [v[0] for v in verts] 
    y = [v[1] for v in verts] 
    
    
    
    p = figure(title="Top 3 friends and enemies of the subreddit "+subreddit,plot_width=600, plot_height=600)
    text = friend_names+enemies_names+nothing

    source = ColumnDataSource({'x':x + [centre ],'y':y + [1],'text':text})

    p.line(x="x", y="y", source=source)

    labels = LabelSet(x="x",y="y",text="text",source=source)

    p.add_layout(labels)
    
    p.title.text_color = "green"
    p.title.text_font_size = "15px"

    # example factor:
    f1=np.array(friend_toxicity+enemies_toxicity)
    f2=np.array(friends_positivity+enemies_positivity)

    #xt = np.array(x)
    flist=[f1,f2]
    
    labels=['Toxicity','Positivity']
    
    colors = ['red','green']
    
    for i in range(len(flist)):
        xt, yt = radar_patch(flist[i], theta, centre)
        p.patch(x=xt, y=yt, fill_alpha=0.15, fill_color=colors[i],legend_label=labels[i])
    show(p)

In [60]:
radar_graph('the_donald')

# Option 2

In [61]:
def radar_graph2(subreddit):    
    a=df[df['SOURCE_SUBREDDIT']==subreddit]
    a=a.sort_values('toxicity')
    
    
    #TOXICITY
    friend_toxicity=list(a['toxicity'][:3])
    enemies_toxicity=list(a['toxicity'][-3:])
    
    #POSITIVITY
    friends_positivity=list(a['positive_rate'][:3])
    enemies_positivity=list(a['positive_rate'][-3:])
    
    #NORM_TOXICITY
    friend_norm_toxicity=list(a['norm_toxicity'][:3])
    enemies_norm_toxicity=list(a['norm_toxicity'][-3:])
    
    
    
    #TOP 3 friends
    friend_names=list(a['TARGET_SUBREDDIT'][:3])
    
    #TOP 3 enemies
    enemies_names=list(a['TARGET_SUBREDDIT'][-3:])
    
    
    
    ####CREATE RADAR GRAPH
    
    nothing=[""]
    
    #number of vertices
    num_vars = 3

    centre = 0.5

    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2

    def unit_poly_verts(theta, centre ):
        """Return vertices of polygon for subplot axes.
        This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
        """
        x0, y0, r = [centre ] * 3
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts

    def radar_patch(r, theta, centre ):
        """ Returns the x and y coordinates corresponding to the magnitudes of 
        each variable displayed in the radar plot
        """
        # offset from centre of circle
        offset = 0.0
        yt = (r*centre + offset) * np.sin(theta) + centre 
        xt = (r*centre + offset) * np.cos(theta) + centre 
        return xt, yt

    verts = unit_poly_verts(theta, centre)
    x = [v[0] for v in verts] 
    y = [v[1] for v in verts] 
    
    
    p = figure(title="Top 3 friends and enemies of the subreddit "+subreddit,plot_width=600, plot_height=600)
    text = ['positive_rate','toxicity' ,'norm_toxicity',]+nothing

    source = ColumnDataSource({'x':x + [centre ],'y':y + [1],'text':text})

    p.line(x="x", y="y", source=source)

    labels = LabelSet(x="x",y="y",text="text",source=source)

    p.add_layout(labels)
    
    p.title.text_color = "orange"
    p.title.text_font_size = "15px"

    # example factor: #pos. tox. norm_tox
    #f1=np.array([0.5,0.6,0.7])
    f1=np.array([friends_positivity[0],friend_toxicity[0],friend_norm_toxicity[0]])
    f2=np.array([friends_positivity[1],friend_toxicity[1],friend_norm_toxicity[1]])
    f3=np.array([friends_positivity[2],friend_toxicity[2],friend_norm_toxicity[2]])
    f4=np.array([enemies_positivity[0],enemies_toxicity[0],enemies_norm_toxicity[0]])
    f5=np.array([enemies_positivity[1],enemies_toxicity[1],enemies_norm_toxicity[1]])
    f6=np.array([enemies_positivity[2],enemies_toxicity[2],enemies_norm_toxicity[2]])
    

    #xt = np.array(x)
    flist=[f1,f2,f3,f4,f5,f6]
    
    colors = ['red','green','blue','pink','orange','grey']
    
    labels=friend_names+enemies_names
    
    for i in range(len(flist)):
        xt, yt = radar_patch(flist[i], theta, centre)
        p.patch(x=xt, y=yt, fill_alpha=0.15, fill_color=colors[i],legend_label=labels[i])
    show(p)

In [62]:
radar_graph2('the_donald')